In [ ]:
import datetime
from dateutil.parser import parse
import re
import pandas as pd

In [ ]:
def parse_float(s):
    f = []
    is_negative = False
    try:
        s = s.strip()
        if "-" in s or "−" in s or "âˆ’" in s:
            is_negative = True
        # print(f"is_negative={is_negative}")
        f = re.findall(r"[-+]?\d*\.\d+|\d+", s.replace("$", "").replace(",", ""))
        # print(f"s={s},  f={f}")
    except:
        pass
    if not f:
        return None
    return -1*float(f[0]) if is_negative else float(f[0])

In [ ]:
amt = "$2,646.92" # "âˆ’$2,646.92" # " −$1,908.08"  # "-$624.39" # "−$624.39" #
parse_float(amt)

In [ ]:
file_txt = "all-transactions-20211231.txt"

In [ ]:
with open(file_txt) as f:
    data = f.read().split("\n")

In [ ]:
len(data), data

In [ ]:
headers = """Date	
Amount
Cardmember	
Description""".split()

In [ ]:
headers

In [ ]:
expenses = []
payments = []
for i in range(0, len(data), 4):
    # print(f"i={i}:  {','.join(data[i:i+4])}")
    dt = parse(data[i]).strftime("%Y-%m-%d")
    amt = parse_float(data[i+3])
    spender = data[i+1]
    desc = data[i+2]
    if "AUTOPAY" in desc:
        # print(f"amt= {amt},  raw= {data[i+3]}")
        payments.append([dt, amt, spender, desc])
    else:
        expenses.append([dt, amt, spender, desc])

In [ ]:
payments

In [ ]:
expenses

In [ ]:
df_pay = pd.DataFrame(data=payments, columns=headers)

In [ ]:
df_pay

In [ ]:
df_exp = pd.DataFrame(data=expenses, columns=headers)

In [ ]:
df_exp

In [ ]:
df_pay.groupby(['Cardmember'])['Amount'].sum()

In [ ]:
df_exp.groupby(['Cardmember'])['Amount'].sum()

In [ ]:
file_xlsx = file_txt.split(".")[0] + ".xlsx"

file_xlsx

In [ ]:
writer = pd.ExcelWriter(file_xlsx, engine='xlsxwriter')

In [ ]:
df_exp.to_excel(writer, sheet_name='Expenses', index=False)
df_pay.to_excel(writer, sheet_name='Payments', index=False)

In [ ]:
writer.save()